In [1]:
# from flair.models import TextClassifier
# from flair.data import Sentence
# classifier = TextClassifier.load('en-sentiment')
# sentence = Sentence('ache is bad!')
# classifier.predict(sentence)
# # print sentence with predicted labels
# print('Sentence above is: ', sentence.labels)

In [2]:
import pandas as pd
import numpy as np
import itertools
import codecs
import os

In [3]:
# get the current working directory
data_path = '/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/CSV/'
# os.path.abspath(os.path.join(os.path.dirname( '__file__' ), '..', 'Datasets'))+'/'

# Although the dataset says csv, it is tab delimited. In addition to this, they have severe codels problems. 
# So best to parse throught codes first. 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 2: invalid start byte

#open for reading with "universal" type set

import codecs

doc_d_t = codecs.open(data_path+'GD_extracts.csv','rU','UTF-8') 
GDA_temp = pd.read_csv(doc_d_t, sep='\t', na_filter = False, names=['text', 'label'], index_col = False)


In [4]:
df = GDA_temp[['label','text']]
df = df[df['text'].apply(lambda x: len(x)>50)] 
# df.reset_index(inplace=True)
df

,label,text
0,NGD,Sucrose- and H+-Dependent Charge Movements Ass...
1,NGD,Conceived and designed the experiments: AC HK ...
2,NGD,Associations of Amylin with Inflammatory Marke...
3,NGD,Conceived and designed the experiments: YL XL ...
5,NGD,GR3027 antagonizes GABAA receptor-potentiating...
...,...,...
70231,NGD,Identical or similar (difference in the presen...
70232,NGD,"On the other hand, all IgG anti-HIV evaluated ..."
70233,NGD,"Notably, we could not find an association betw..."
70234,NGD,Representative IgG anti-HIV patterns from two ...


In [5]:
labels_count = df['label'].value_counts()
labels_count.plot(kind="bar")
print(df['label'].value_counts())

NGD    41203
YGD      429
Name: label, dtype: int64


In [6]:
# Since the data GD association data is so less. We need to bump the data from the publicly available datasets.

YGD_df = df[df['label']=='YGD']
NGD_df = df[df['label']=='NGD']

In [12]:
data_path_ = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/'

doc_d_t = codecs.open(data_path_+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_target'+'.csv','rU','UTF-8') 
EUADR_drug_target = pd.read_csv(doc_d_t, sep='\t', na_filter = False)
EUADR_drug_target['label'] = 'NGD'
EUADR_NGD_df = EUADR_drug_target[['SENTENCE','label']]

doc_t_d = codecs.open(data_path_+'EUADR_Corpus_IBIgroup/'+'EUADR_target_disease'+'.csv','rU','UTF-8',errors='ignore') 
EUADR_target_disease = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
EUADR_target_disease['label'] = 'YGD'
EUADR_YGD_df = EUADR_target_disease[['SENTENCE','label']]

EUADR_YGD_df.rename(columns={"SENTENCE": "text"}, inplace=True)
EUADR_YGD_df.reset_index(inplace=True)

EUADR_NGD_df.rename(columns={"SENTENCE": "text"}, inplace=True)
EUADR_NGD_df.reset_index(inplace=True)

In [13]:
doc_t_d = codecs.open(data_path_+'GAD_Corpus_IBIgroup/'+'GAD_Y_N'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_Y_N = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_Y_N['label'] = 'YGD'

doc_t_d = codecs.open(data_path_+'GAD_Corpus_IBIgroup/'+'GAD_F'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_F = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_F['label'] = 'YGD'

GAD_YGD_df = GAD_target_disease_Y_N[['GAD_CONCLUSION', 'label']].append(GAD_target_disease_F[['GAD_CONCLUSION', 'label']])


GAD_YGD_df.rename(columns={"GAD_CONCLUSION": "text"}, inplace=True)

GAD_YGD_df.reset_index(inplace=True)

In [16]:
YGD_combined = YGD_df.append(EUADR_YGD_df.append(GAD_YGD_df))
YGD_combined.to_csv(data_path+'YGD.csv', sep = '\t')

In [18]:
NGD_combined = EUADR_NGD_df.append(NGD_df)
NGD_combined_ = NGD_combined.head(7000)
NGD_combined_.to_csv(data_path+'NGD.csv', sep = '\t')

In [22]:
# Load dataset

doc_d_t = codecs.open(data_path+'Association_Dataset.csv','rU','UTF-8') 
GDA = pd.read_csv(doc_d_t, sep='\t', na_filter = False, index_col = False)



In [23]:
GDA.head(10)

,label,text
0,YGD,"However, the role of amylin in obesity related..."
1,YGD,"Therefore, we aimed to evaluate the distributi..."
2,YGD,The amylin-MetS association is independent of ...
3,YGD,"Amylin, also named islet amyloid polypeptide (..."
4,YGD,Chronic infusion of amylin into the brain redu...
5,YGD,Elevated levels of amylin were observed in obe...
6,YGD,We also examined whether the amylin-MetS assoc...
7,YGD,"The severity of MetS, indicated as ≤1, 2, 3 an..."
8,YGD,These results suggest that amylin is associate...
9,YGD,No significant interactions were observed betw...


In [27]:
# shuffle rows in random
data = GDA.sample(frac=1).reset_index(drop=True)
data.head(10)

,label,text
0,NGD,Scaffolds were successfully fabricated using t...
1,YGD,The results showed that the triplet repeat pol...
2,YGD,We conclude that ACVR2B mutations are present...
3,NGD,The benzoic acid-based flavonoid analog CAF-40...
4,YGD,HLA-DQB1*0401 is a useful marker for determini...
5,NGD,Higher levels of collagen I and collagen X are...
6,NGD,There are several ways to deal with this depen...
7,NGD,This was done however to simplify the process ...
8,NGD,The duplex energy was normalized by the energy...
9,YGD,Our findings suggest that IMPA2 or a gene near...


In [29]:
data_loc = '/nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/processed_datasets/'

data['label'] = '__label__' + data['label'].astype(str)
data.iloc[0:int(len(data)*0.8)].to_csv(data_loc+'train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv(data_loc+'test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv(data_loc+'dev.csv', sep='\t', index = False, header = False);


In [30]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.embeddings import TokenEmbeddings, PooledFlairEmbeddings, StackedEmbeddings, CharacterEmbeddings
# from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from typing import List


elmo_embeddings_pubmed = ELMoEmbeddings('pubmed')


In [31]:
corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_loc), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


2019-09-12 15:27:57,834 Reading data from /nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/processed_datasets
2019-09-12 15:27:57,835 Train: /nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/processed_datasets/train.csv
2019-09-12 15:27:57,835 Dev: /nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/processed_datasets/dev.csv
2019-09-12 15:27:57,836 Test: /nfs/gns/literature/Santosh_Tirunagari/EBI standard Dataset/processed_datasets/test.csv


/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  """Entry point for launching an IPython kernel.
/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/flair/data_fetcher.py:463: Depre

In [32]:
# 3. make a list of word embeddings
word_embeddings = [WordEmbeddings('glove'),
                   # comment in flair embeddings for state-of-the-art results
                   FlairEmbeddings('news-forward'),
                   FlairEmbeddings('news-backward'),
                   CharacterEmbeddings(), 
                   PooledFlairEmbeddings('pubmed-backward', pooling='max'),
                   PooledFlairEmbeddings('pubmed-forward', pooling='max'),
                   # BertEmbedding('bert-large-cased'),
                   elmo_embeddings_pubmed
                   ]



In [33]:
# document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                   rnn_type = "LSTM",
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

In [34]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)



2019-09-12 15:28:37,556 Computing label dictionary. Progress:


100%|██████████| 10491/10491 [00:00<00:00, 323909.39it/s]

2019-09-12 15:28:37,594 [b'NGD', b'YGD']


In [35]:
trainer = ModelTrainer(classifier, corpus)


In [36]:
# 7. start the training
# trainer.train(data_loc, max_epochs=10)

result_loc = '../flair_models/text_classification/association_model'
trainer.train(result_loc,
              learning_rate=0.15,
              mini_batch_size=128,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

2019-09-12 15:29:02,593 ----------------------------------------------------------------------------------------------------
2019-09-12 15:29:02,594 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
      (list_embedding_3): CharacterEmbeddings(
        (char_embedding): Embedding(275, 25)
        (char_rnn): LSTM(25, 25, bidirectional=True)
 

2019-09-12 15:50:52,365 epoch 4 - iter 40/82 - loss 0.06727652 - samples/sec: 42.07
2019-09-12 15:51:18,850 epoch 4 - iter 48/82 - loss 0.07068225 - samples/sec: 41.40
2019-09-12 15:51:45,632 epoch 4 - iter 56/82 - loss 0.06951936 - samples/sec: 41.43
2019-09-12 15:52:13,157 epoch 4 - iter 64/82 - loss 0.07084347 - samples/sec: 40.61
2019-09-12 15:52:40,534 epoch 4 - iter 72/82 - loss 0.07085819 - samples/sec: 40.41
2019-09-12 15:53:08,692 epoch 4 - iter 80/82 - loss 0.07135021 - samples/sec: 38.66
2019-09-12 15:53:21,484 ----------------------------------------------------------------------------------------------------
2019-09-12 15:53:21,496 EPOCH 4 done: loss 0.0710 - lr 0.1500
2019-09-12 15:53:36,833 DEV : loss 0.08334286510944366 - score 0.9733
2019-09-12 15:53:37,252 BAD EPOCHS (no improvement): 2
2019-09-12 15:53:37,259 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embedd

2019-09-12 16:21:33,781 epoch 10 - iter 8/82 - loss 0.03162813 - samples/sec: 38.00
2019-09-12 16:22:02,149 epoch 10 - iter 16/82 - loss 0.03699215 - samples/sec: 38.64
2019-09-12 16:22:29,832 epoch 10 - iter 24/82 - loss 0.07051782 - samples/sec: 39.12
2019-09-12 16:22:57,660 epoch 10 - iter 32/82 - loss 0.06700781 - samples/sec: 40.51
2019-09-12 16:23:28,234 epoch 10 - iter 40/82 - loss 0.06170231 - samples/sec: 36.94
2019-09-12 16:23:57,531 epoch 10 - iter 48/82 - loss 0.05814087 - samples/sec: 39.39
2019-09-12 16:24:25,985 epoch 10 - iter 56/82 - loss 0.05527805 - samples/sec: 39.39
2019-09-12 16:24:54,011 epoch 10 - iter 64/82 - loss 0.05220671 - samples/sec: 40.37
2019-09-12 16:25:22,620 epoch 10 - iter 72/82 - loss 0.05148823 - samples/sec: 41.74
2019-09-12 16:25:52,579 epoch 10 - iter 80/82 - loss 0.05095495 - samples/sec: 37.92
2019-09-12 16:26:04,080 ----------------------------------------------------------------------------------------------------
2019-09-12 16:26:04,092 EP

2019-09-12 16:52:31,576 EPOCH 15 done: loss 0.0170 - lr 0.0750
2019-09-12 16:52:46,803 DEV : loss 0.09698867797851562 - score 0.9794
2019-09-12 16:52:47,220 BAD EPOCHS (no improvement): 2
2019-09-12 16:52:47,227 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-12 16:52:51,191 epoch 16 - iter 0/82 - loss 0.02773589 - samples/sec: 278.63
2019-09-12 16:53:20,210 epoch 16 - iter 8/82 - loss 0.01899114 - samples/sec: 38.82
2019-09-12 16:53:48,898 epoch 16 - iter 16/82 - loss 0.01829314 - samples/sec: 38.52
2019-09-12 16:54:17,107 epoch 16 - iter 24/82 - loss 0.01698560 - samples/sec: 39.16
2019-09-12 16:54:48,288 epoch 16 - iter 32/82 - loss 0.01605895 - samples/sec: 35.20
2019-09-12 16:55:17,002 epoch 16 - iter 40/82 - loss 0.01590641 - samples/sec: 38.02
2019-09-12 16:55:45,321 epoch 16 - iter 48/82 - loss 0.01581049 - samples/sec: 38.97
2019-09-12 16:56:13,267 epoch

2019-09-12 17:23:09,553 epoch 21 - iter 64/82 - loss 0.01234696 - samples/sec: 39.39
2019-09-12 17:23:39,757 epoch 21 - iter 72/82 - loss 0.01220834 - samples/sec: 35.56
2019-09-12 17:24:08,930 epoch 21 - iter 80/82 - loss 0.01220218 - samples/sec: 36.96
2019-09-12 17:24:19,569 ----------------------------------------------------------------------------------------------------
2019-09-12 17:24:19,579 EPOCH 21 done: loss 0.0121 - lr 0.0750
2019-09-12 17:24:34,940 DEV : loss 0.10547951608896255 - score 0.9794
2019-09-12 17:24:35,350 BAD EPOCHS (no improvement): 5
2019-09-12 17:24:35,355 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-12 17:24:39,546 epoch 22 - iter 0/82 - loss 0.01562314 - samples/sec: 267.97
2019-09-12 17:25:08,433 epoch 22 - iter 8/82 - loss 0.00849163 - samples/sec: 37.09
2019-09-12 17:25:36,025 epoch 22 - iter 16/82 - loss 0.00880961 - samples/

...

2019-09-12 20:51:08,262 epoch 61 - iter 56/82 - loss 0.00397226 - samples/sec: 39.29
2019-09-12 20:51:34,486 epoch 61 - iter 64/82 - loss 0.00359355 - samples/sec: 40.94
2019-09-12 20:52:00,567 epoch 61 - iter 72/82 - loss 0.00340244 - samples/sec: 41.18
2019-09-12 20:52:32,522 epoch 61 - iter 80/82 - loss 0.00323551 - samples/sec: 33.82
2019-09-12 20:52:43,330 ----------------------------------------------------------------------------------------------------
2019-09-12 20:52:43,342 EPOCH 61 done: loss 0.0032 - lr 0.0012
2019-09-12 20:52:59,088 DEV : loss 0.12187598645687103 - score 0.9825
2019-09-12 20:52:59,573 BAD EPOCHS (no improvement): 3
2019-09-12 20:52:59,583 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-12 20:53:03,423 epoch 62 - iter 0/82 - loss 0.00131792 - samples/sec: 291.63
2019-09-12 20:53:31,469 epoch 62 - iter 8/82 - loss 0.00117187 - samples/

2019-09-12 21:19:07,376 epoch 67 - iter 16/82 - loss 0.00452538 - samples/sec: 41.81
2019-09-12 21:19:34,030 epoch 67 - iter 24/82 - loss 0.00536264 - samples/sec: 40.07
2019-09-12 21:20:00,389 epoch 67 - iter 32/82 - loss 0.00448346 - samples/sec: 41.59
2019-09-12 21:20:26,851 epoch 67 - iter 40/82 - loss 0.00432685 - samples/sec: 41.01
2019-09-12 21:20:54,003 epoch 67 - iter 48/82 - loss 0.00382817 - samples/sec: 40.20
2019-09-12 21:21:23,703 epoch 67 - iter 56/82 - loss 0.00341681 - samples/sec: 36.60
2019-09-12 21:21:49,557 epoch 67 - iter 64/82 - loss 0.00328504 - samples/sec: 42.68
2019-09-12 21:22:16,209 epoch 67 - iter 72/82 - loss 0.00308406 - samples/sec: 40.68
2019-09-12 21:22:45,104 epoch 67 - iter 80/82 - loss 0.00292332 - samples/sec: 37.94
2019-09-12 21:22:56,413 ----------------------------------------------------------------------------------------------------
2019-09-12 21:22:56,423 EPOCH 67 done: loss 0.0029 - lr 0.0006
2019-09-12 21:23:11,523 DEV : loss 0.1221872717

2019-09-12 21:48:01,099 DEV : loss 0.12241408228874207 - score 0.9825
2019-09-12 21:48:01,514 BAD EPOCHS (no improvement): 2
2019-09-12 21:48:01,566 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-12 21:48:05,592 epoch 73 - iter 0/82 - loss 0.00024149 - samples/sec: 273.88
2019-09-12 21:48:34,130 epoch 73 - iter 8/82 - loss 0.00165481 - samples/sec: 37.57
2019-09-12 21:49:00,798 epoch 73 - iter 16/82 - loss 0.00494965 - samples/sec: 40.38
2019-09-12 21:49:27,325 epoch 73 - iter 24/82 - loss 0.00374174 - samples/sec: 40.94
2019-09-12 21:49:52,876 epoch 73 - iter 32/82 - loss 0.00313344 - samples/sec: 42.04
2019-09-12 21:50:19,237 epoch 73 - iter 40/82 - loss 0.00313797 - samples/sec: 40.88
2019-09-12 21:50:48,772 epoch 73 - iter 48/82 - loss 0.00282015 - samples/sec: 36.56
2019-09-12 21:51:15,548 epoch 73 - iter 56/82 - loss 0.00270737 - samples/sec: 40.50
2019-09

2019-09-12 22:16:41,141 epoch 78 - iter 64/82 - loss 0.00263985 - samples/sec: 41.71
2019-09-12 22:17:07,600 epoch 78 - iter 72/82 - loss 0.00312511 - samples/sec: 40.42
2019-09-12 22:17:35,969 epoch 78 - iter 80/82 - loss 0.00294384 - samples/sec: 38.10
2019-09-12 22:17:46,667 ----------------------------------------------------------------------------------------------------
2019-09-12 22:17:46,678 EPOCH 78 done: loss 0.0030 - lr 0.0001
2019-09-12 22:18:01,508 DEV : loss 0.1225278303027153 - score 0.9825
2019-09-12 22:18:01,978 BAD EPOCHS (no improvement): 2
2019-09-12 22:18:01,984 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-12 22:18:05,719 epoch 79 - iter 0/82 - loss 0.00356002 - samples/sec: 300.75
2019-09-12 22:18:33,628 epoch 79 - iter 8/82 - loss 0.00205756 - samples/sec: 38.54
2019-09-12 22:19:00,612 epoch 79 - iter 16/82 - loss 0.00217836 - samples/s

{'test_score': 0.9779,
 'dev_score_history': [0.955,
  0.9748,
  0.9718,
  0.9733,
  0.9726,
  0.968,
  0.9802,
  0.971,
  0.9779,
  0.9634,
  0.9771,
  0.9794,
  0.9779,
  0.9794,
  0.9794,
  0.9817,
  0.9809,
  0.9817,
  0.9787,
  0.9802,
  0.9794,
  0.9802,
  0.9817,
  0.9825,
  0.9802,
  0.9802,
  0.9817,
  0.984,
  0.9825,
  0.9802,
  0.9817,
  0.9802,
  0.984,
  0.9802,
  0.984,
  0.9809,
  0.9832,
  0.9832,
  0.9832,
  0.984,
  0.984,
  0.984,
  0.984,
  0.984,
  0.9832,
  0.9832,
  0.9825,
  0.9825,
  0.9832,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825,
  0.9825],
 'train_loss_history': [0.24342985887353014,
  0.10240639482692974,
  0.07758304274572832,
  0.0710099162988183,
  0.06490829710780484,
  0.051419

In [ ]:
import torch
torch.cuda.is_available()

In [37]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load(result_loc+'/best-model.pt')


2019-09-13 09:56:10,059 loading file ../flair_models/text_classification/association_model/best-model.pt


In [47]:
# sentence = Sentence('The FCRL3 polymorphisms associated with rheumatoid arthritis in a Japanese population are not associated per se with rheumatoid arthritis in a Spanish population.')
# sentence = Sentence('We obtained evidence from two Caucasian study populations that the His298-allele of PTGES2 Arg298His confers to reduced risk of type 2 diabetes.')
# sentence = Sentence('The sensitivity and specificity of leptomeningeal enhancement to identify patients with ABRA+CAA-RI were 70.4% and 92.6%, respectively, whereas the positive LR was 9.5.')
sentence = Sentence('Our findings suggest a role for ANP in PDR')
classifier.predict(sentence)
str(sentence.labels[0]).split()[1]

'YGD'

In [50]:
from tqdm import tqdm
prediction_dataset = '/nfs/gns/literature/Santosh_Tirunagari/Dataset/test.txt'


In [1]:
import csv
import re     
# Open the file with read only permit
f = open(prediction_dataset)
line = f.readline()
while line:
    sentence = Sentence(f.readline())
    classifier.predict(sentence)
    with open('/nfs/gns/literature/Santosh_Tirunagari/Dataset/association_scores.tsv', 'a') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        if str(sentence.labels[0]).split()[0] == 'YGD':
            tsv_writer.writerow([f.readline(), float(str(sentence.labels[0]).split()[1].replace('(', '').replace(')',''))])
        else:
            tsv_writer.writerow([f.readline(), 1.0-float(str(sentence.labels[0]).split()[1].replace('(', '').replace(')',''))])

    
f.close()

    

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

f1_score(ground_truth, predicted_label, average='micro') 